# **(파이토치 트랜스포머를 활용한) 자연어 처리와 컴퓨터비전 심층학습**
## 5장 토큰화 (p230~ p263)

### [토크나이저 모델 학습]

청와대 청원 데이터 사용

In [4]:
!pip install sentencepiece Korpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.3 MB/s eta 0:00:00


In [1]:
# 청와대 청원 데이터 다운로드

from Korpora import Korpora

corpus = Korpora.load("korean_petitions")
dataset = corpus.train
petition = dataset[0]

print("청원 시작일 :", petition.begin)
print("청원 종료일 :", petition.end)
print("청원 동의 수 :", petition.num_agree)
print("청원 범주 :", petition.category)
print("청원 제목 :", petition.title)
print("청원 본문 :", petition.text[:30])


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/petitions_archive
    References :

    청와대 국민청원 게시판의 데이터를 월별로 수집한 것입니다.
    청원은 게시판에 글을 올린 뒤, 한달 간 청원이 진행됩니다.
    수집되는 데이터는 청원종료가 된 이후의 데이터이며, 청원 내 댓글은 수집되지 않습니다.
    단 청원의 동의 개수는 수집됩니다.
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[korean_petitions] download petitions_2017-08: 1.84MB [00:00, 15.1MB/s]                           
[korean_petitions] download petitions_2017-09: 20.4MB [00:00, 59.0MB/s]                            
[korean_petitions] download petitions_2017-10: 12.0MB [00:00, 55.4MB/s]                            
[korean_petitions] download petitions_2017-11: 28.4MB [00:00, 73.2MB/s]                            
[korean_petitions] download petitions_2017-12: 29.0MB [00:00, 103MB/s]                             
[korean_petitions] download petitions_2018-01: 43.9MB [00:00, 151MB/s]                            
[korean_petitions] download petitions_2018-02: 33.8MB [00:00, 147MB/s]                            
[korean_petitions] download petitions_2018-03: 34.3MB [00:00, 137MB/s]                            
[korean_petitions] download petitions_2018-04: 35.5MB [00:00, 142MB/s]                            
[korean_petitions] download petitions_2018-05: 37.5MB [00:00, 147MB/s]                            
[korea

청원 시작일 : 2017-08-25
청원 종료일 : 2017-09-24
청원 동의 수 : 88
청원 범주 : 육아/교육
청원 제목 : 학교는 인력센터, 취업센터가 아닙니다. 정말 간곡히 부탁드립니다.
청원 본문 : 안녕하세요. 현재 사대, 교대 등 교원양성학교들의 예비


In [6]:
# 학습 데이터세트 생성
from Korpora import Korpora

corpus = Korpora.load("korean_petitions")
petitions = corpus.get_all_texts()

with open("/content/corpus.txt", "w", encoding="utf-8") as f:
    for petition in petitions:
        f.write(petition + "\n")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/petitions_archive
    References :

    청와대 국민청원 게시판의 데이터를 월별로 수집한 것입니다.
    청원은 게시판에 글을 올린 뒤, 한달 간 청원이 진행됩니다.
    수집되는 데이터는 청원종료가 된 이후의 데이터이며, 청원 내 댓글은 수집되지 않습니다.
    단 청원의 동의 개수는 수집됩니다.
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `korean_petitions` is already installed at /root/Korpora/korean_petitions/petitions_2017-08
[Korpora] Corpus `korean_petitions` is already installed at /root/Korpora/korean_petitions/petitions_2017-09
[Korpora] Corpus `korean_petitions` is already installed at /root/Korpora/kore

In [7]:
# 토크나이저 모델 학습
from sentencepiece import SentencePieceTrainer

SentencePieceTrainer.Train(
    "--input=/content/corpus.txt "
    "--model_prefix=/content/petition_bpe "
    "--vocab_size=8000 "
    "--model_type=bpe"
)

In [10]:
# 바이트 페어 인코딩 토큰화
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.load("/content/petition_bpe.model")  # <- 경로 수정

sentence = "안녕하세요, 토크나이저가 잘 학습되었군요!"
sentences = ["이렇게 입력값을 리스트로 받아서", "쉽게 토크나이저를 사용할 수 있답니다"]

tokenized_sentence = tokenizer.encode_as_pieces(sentence)
tokenized_sentences = [tokenizer.encode_as_pieces(s) for s in sentences]
print("단일 문장 토큰화 :", tokenized_sentence)
print("여러 문장 토큰화 :", tokenized_sentences)

encoded_sentence = tokenizer.encode_as_ids(sentence)
encoded_sentences = [tokenizer.encode_as_ids(s) for s in sentences]
print("단일 문장 정수 인코딩 :", encoded_sentence)
print("여러 문장 정수 인코딩 :", encoded_sentences)

decode_ids = [tokenizer.decode_ids(ids) for ids in encoded_sentences]
decode_pieces = [tokenizer.decode_pieces(tokenizer.encode_as_pieces(s)) for s in sentences]
print("정수 인코딩에서 문장 변환 :", decode_ids)
print("하위 단어 토큰에서 문장 변환 :", decode_pieces)


단일 문장 토큰화 : ['▁안녕하세요', ',', '▁토', '크', '나', '이', '저', '가', '▁잘', '▁학', '습', '되었', '군요', '!']
여러 문장 토큰화 : [['▁이렇게', '▁입', '력', '값을', '▁리', '스트', '로', '▁받아서'], ['▁쉽게', '▁토', '크', '나', '이', '저', '를', '▁사용할', '▁수', '▁있', '답니다']]
단일 문장 정수 인코딩 : [667, 6553, 994, 6880, 6544, 6513, 6590, 6523, 161, 110, 6554, 872, 787, 6648]
여러 문장 정수 인코딩 : [[372, 182, 6677, 4433, 1772, 1613, 6527, 4162], [1681, 994, 6880, 6544, 6513, 6590, 6536, 5852, 19, 5, 2639]]
정수 인코딩에서 문장 변환 : ['이렇게 입력값을 리스트로 받아서', '쉽게 토크나이저를 사용할 수 있답니다']
하위 단어 토큰에서 문장 변환 : ['이렇게 입력값을 리스트로 받아서', '쉽게 토크나이저를 사용할 수 있답니다']


In [11]:
# 어휘 사전 불러오기
from sentencepiece import SentencePieceProcessor

tokenizer = SentencePieceProcessor()
tokenizer.load("/content/petition_bpe.model")

vocab = {idx: tokenizer.id_to_piece(idx) for idx in range(tokenizer.get_piece_size())}
print(list(vocab.items())[:5])
print("vocab size :", len(vocab))


[(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '니다'), (4, '▁이')]
vocab size : 8000


### [워드피스 토크나이저 학습]

In [12]:
# 워드피스 토크나이저 학습
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.normalizers import Sequence, NFD, Lowercase
from tokenizers.pre_tokenizers import Whitespace
import os

# 저장 폴더 생성
os.makedirs("/content/models", exist_ok=True)

# 토크나이저 정의
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = Sequence([NFD(), Lowercase()])
tokenizer.pre_tokenizer = Whitespace()

# WordPiece 트레이너 정의
trainer = WordPieceTrainer(vocab_size=8000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

# 학습
tokenizer.train(files=["/content/corpus.txt"], trainer=trainer)

# 저장
tokenizer.save("/content/models/petition_wordpiece.json")


In [13]:
# 워드피스 토큰화
from tokenizers import Tokenizer
from tokenizers.decoders import WordPiece as WordPieceDecoder

# 토크나이저 로드
tokenizer = Tokenizer.from_file("/content/models/petition_wordpiece.json")
tokenizer.decoder = WordPieceDecoder()

# 테스트 문장
sentence = "안녕하세요, 토크나이저가 잘 학습되었군요!"
sentences = ["이렇게 입력값을 리스트로 받아서", "쉽게 토크나이저를 사용할 수 있답니다"]

# 단일 문장 인코딩
encoded_sentence = tokenizer.encode(sentence)
# 여러 문장 인코딩
encoded_sentences = tokenizer.encode_batch(sentences)

# 출력
print("인코더 형식 :", type(encoded_sentence))
print("단일 문장 토큰화 :", encoded_sentence.tokens)
print("여러 문장 토큰화 :", [enc.tokens for enc in encoded_sentences])
print("단일 문장 정수 인코딩 :", encoded_sentence.ids)
print("여러 문장 정수 인코딩 :", [enc.ids for enc in encoded_sentences])
print("정수 인코딩에서 문장 변환 :", tokenizer.decode(encoded_sentence.ids))

인코더 형식 : <class 'tokenizers.Encoding'>
단일 문장 토큰화 : ['안', '##ᄂ', '##ᅧᆼ', '##하', '##세요', ',', 'ᄐ', '##ᅩ', '##ᄏ', '##ᅳ', '##나', '##이', '##저', '##가', '잘', '학', '##스', '##ᆸ', '##되', '##었', '##군', '##요', '!']
여러 문장 토큰화 : [['이러', '##ᇂ게', '입', '##력', '##가', '##ᆹ', '##을', 'ᄅ', '##ᅵ', '##스', '##트', '##로', '받', '##아', '##서'], ['ᄉ', '##ᅱ', '##ᆸ', '##게', 'ᄐ', '##ᅩ', '##ᄏ', '##ᅳ', '##나', '##이', '##저', '##를', '사', '##용', '##할', '수', '있', '##다', '##ᆸ니다']]
단일 문장 정수 인코딩 : [7625, 4229, 7508, 7487, 7723, 16, 313, 4224, 4281, 4230, 7538, 7480, 7541, 7493, 7847, 7779, 7517, 4251, 7562, 7734, 7963, 7520, 5]
여러 문장 정수 인코딩 : [[7910, 7755, 7876, 7693, 7493, 4285, 7490, 302, 4226, 7517, 7758, 7500, 7706, 7547, 7505], [306, 4255, 4251, 7523, 313, 4224, 4281, 4230, 7538, 7480, 7541, 7515, 7533, 7613, 7658, 7567, 7531, 7483, 7494]]
정수 인코딩에서 문장 변환 : 안녕하세요, 토크나이저가 잘 학습되었군요!
